In [1]:
import argparse
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import transforms
from copy import deepcopy

This K-means implementation is much faster than sklearn version. It uses vectorization... test it yourself.

In [176]:
class k_means(object):
    def __init__(self, data, num_clusters, normalize_data = True, rnd_seed = 42):
        self.data = data
        self.clusters = np.array([i for i in range(num_clusters)])
        if num_clusters > 10:
            print("This class does not support more than 10 clusters!")
            sys.exit()
        if normalize_data:
          self.normalize()
          print("Data normalized: x = (x-x_mean)/x_dev")

        # Draw random data points and make them centroids
        np.random.seed(rnd_seed)
        self.centroids = self.data[np.array(np.round(len(self.data)*np.random.sample(num_clusters)), np.int)]
        print("Cluster labels: ")
        print(self.clusters)
        print("Initial centroids: ")
        print(self.centroids)

        # Define colors for 9 possible classes on map drawing...
        self.colors = {1:[25,25,112], 2:[100,149,237], 3:[255,70,0], 4: [255,0,255], 
                        5:[255,255,0], 6:[0,128,128], 7:[148,0,211], 
                        8:[255,20,147], 9:[220,20,60], 10:[128,0,0]}

    def normalize(self):
        for i in range(len(self.data[0,:])):
            mean = np.mean(self.data[:,i])
            std = np.std(self.data[:,i])
            self.data[:,i] = (self.data[:,i] - mean)/std
        return True

    def solve(self, max_iterations):
        for j in range(max_iterations):
            distances = self.calc_distance()  
            clusters = np.argmin(distances, axis=1) # get coef. of minimum distance
            clustered_data = np.zeros([len(self.data),])
            for i in range(len(self.data)):
                if clusters[i] == 0:
                    clustered_data[i] = 0
                elif clusters[i] == 1:
                    clustered_data[i] = 1
                elif clusters[i] == 2:
                    clustered_data[i] = 2
                    if len(self.clusters) == 3:
                        continue
                elif clusters[i] == 3:
                    clustered_data[i] = 3
                    if len(self.clusters) == 4:
                        continue
                elif clusters[i] == 4:
                    clustered_data[i] = 4
                    if len(self.clusters) == 5:
                        continue
                elif clusters[i] == 5:
                    clustered_data[i] = 5
                    if len(self.clusters) == 6:
                        continue
                elif clusters[i] == 6:
                    clustered_data[i] = 6
                    if len(self.clusters) == 7:
                        continue
                elif clusters[i] == 7:
                    clustered_data[i] = 7
                    if len(self.clusters) == 8:
                        continue
                elif clusters[i] == 8:
                    clustered_data[i] = 8
                    if len(self.clusters) == 9:
                        continue
                elif clusters[i] == 9:
                    clustered_data[i] = 9
                    if len(self.clusters) == 10:
                        continue
                else:
                    clustered_data[i] = 10
            curr_centroids = deepcopy(self.centroids)
            self.calculate_centroids(clustered_data)
            mae = np.sum(np.absolute(np.subtract(curr_centroids, self.centroids)))
            if mae < 0.002:     # Stop the clustering when change on centroids is very small
                break
        print("Final centroids: ")
        print(self.centroids)
        self.clust_data = clustered_data
        return None
            
    def calculate_centroids(self, clustered_data):
        for i in range(len(self.centroids)):
            centroid_ind = np.where(clustered_data==i)
            centroid_data = self.data[centroid_ind]
            self.centroids[i, 0] = np.mean(centroid_data[:,0])
            self.centroids[i, 1] = np.mean(centroid_data[:,1])
        return None


    def plot_result(self, labelx, labely, fig_title,
                    fig_name, map_shape = (144, 72)):
        plt.xlabel(str(labelx))
        plt.ylabel(str(labely))
        plt.title(str(fig_title))
        fig_legend = []
        img_plt = np.zeros([len(self.clust_data), 3], dtype=np.uint8)
        for i in range(len(self.clusters)):
            cent_ind = np.where(self.clust_data == i)
            centroid_data = self.data[cent_ind]
            plt.scatter(centroid_data[:,0], centroid_data[:,1], 
                        s=0.1, color=np.array(self.colors[i+1])/255.0)
            fig_legend.append(str("Cluster ") + str(i))
            img_plt[cent_ind] =np.array(self.colors[i+1])
        img_plt = np.reshape(img_plt, (map_shape[0], map_shape[1], 3))
        img_plt = np.rot90(img_plt)
        fig_legend.append('Centroids')
        plt.scatter(self.centroids[:,0], self.centroids[:,1],
                    s=100, marker='+', c='black')
        plt.legend(fig_legend)
        plt.savefig(str(fig_name), format='png', dpi=200)
        plt.show()
        self.plot_map(img_plt, fig_title, fig_name)
        return None
    
    def plot_map(self, datax, fig_title, fig_name, map_color='x', intpol = 'bilinear'):
        plt.title(str(fig_title))
        im = plt.imshow(datax/255.0)
        plt.savefig(str('map_')+str(fig_name), format='png', dpi=200)
        plt.show()
        return None

    def calc_distance(self):
        ''' Calculate distances of data from all centroids
            and return a matrix, with a column, which is 
            corresponding distance of the point.
        '''
        cdist= []
        for centroid in self.centroids:
            cdist.append(self.get_dist(centroid))
        distances = cdist[0]
        for i in range(1,len(cdist)):
            distances = np.column_stack((distances, cdist[i]))
        return distances

    def silhouette(self):
        S_i = []
        for i in range(len(self.data)):
          point_cl = self.clust_data[i]       # Get cluster of a_i
          point = self.data[i]                # Get location of point_i
          a_i = self.calc_a(point, point_cl)  # Calculate a_i
          b_i = self.calc_b(point, point_cl)  # Calculate b_i
          S_i.append((b_i - a_i)/max(b_i, a_i))
        return np.mean(S_i)

    def plot_silhouette(self, sil_array, title, fig_name):
        plt.plot(np.arange(2, len(sil_array)+2, 1), sil_array)
        plt.xlabel(str("Clusters"))
        plt.ylabel(str("Silhouette Coef."))
        plt.title(str(title))
        plt.savefig(str(fig_name), format='png', dpi=200)
        plt.show()
        return None
    
    def calc_a(self, point, cluster_):
        ''' a_i for Silhouette calculation
        '''
        clust_ind = np.where(self.clust_data == cluster_)
        cluster = self.data[clust_ind]
        return np.nanmean(self.get_dist2(point, cluster))

    def calc_b(self, point, cluster_):
        ''' b_i for Silhouette calculation
        '''
        cluster_id = self.get_closest_centroid(point, cluster_)  # Get the closest cluster id
        clust_ind = np.where(self.clust_data == cluster_id) # Get indexes of the data of that cluster
        cluster = self.data[clust_ind]                      # Get data of that cluster
        return np.nanmean(self.get_dist2(point, cluster))   # Calculate mean distance

    def get_closest_centroid(self, point, cluster):
        cluster_id = 100000000
        dist_min = np.inf
        for i, centroid in enumerate(self.centroids):
          if i != cluster:
             dist1 = (point- centroid)**2
             dist =  np.sum(dist1)**0.5
             if dist < dist_min:
                dist_min = dist
                cluster_id = i
        return cluster_id

    def get_dist2(self, point, xdata):
        point_vect = np.tile(point, (len(xdata),1)) # repeate the vector to calculate all at once
        dist1 = ((point_vect-xdata)**2)
        dist = (dist1[:,0] + dist1[:,1])**0.5       # change this to make it multidimensional
        return dist

    def get_dist(self, centroid):
        ''' Calculate distance of data from centroid
        '''
        centroid_vect = np.tile(centroid, (len(self.data),1)) # repeate the vector to calculate all at once
        dist1 = ((centroid_vect-self.data)**2)
        dist = (dist1[:,0] + dist1[:,1])**0.5       # change this to make it multidimensional
        return dist

In [ ]:
km = k_means(data, num_clusters, normalize_data = True)
km.solve(max_iterations=10)